In [309]:
import numpy as np
import pandas as pd
import math
import sys

In [310]:
def find_best_split(candidate_splits):
    
    best_split = max(candidate_splits, key = lambda x: x[3])
    return best_split[0], best_split[1]

In [311]:
def logbase2(n):
    if n != 0:
        return math.log(n, 2)
    else:
        return -sys.float_info.max

In [312]:
def calculate_entropy(a, b):
    
    total = a + b
    
    prob_a = a/total
    prob_b = b/total
    
    entropy = - (prob_a * logbase2(prob_a)) - (prob_b * logbase2(prob_b))
    
    return entropy

In [313]:
def calculate_info_gain_and_split_info(total_number_of_zeros, total_number_of_ones, number_of_zeros_left, number_of_ones_left, number_of_zeros_right, number_of_ones_right):

    entropy_y = calculate_entropy(total_number_of_zeros, total_number_of_ones)
    entropy_y_left = calculate_entropy(number_of_zeros_left, number_of_ones_left)
    entropy_y_right = calculate_entropy(number_of_zeros_right, number_of_ones_right)

    total_zeros_and_ones_left = number_of_zeros_left + number_of_ones_left
    total_zeros_and_ones_right = number_of_zeros_right + number_of_ones_right

    weight_left = total_zeros_and_ones_left / (total_number_of_zeros + total_number_of_ones)
    weight_right = total_zeros_and_ones_right / (total_number_of_zeros + total_number_of_ones)

    info_gain = entropy_y - (weight_left * entropy_y_left) - (weight_right * entropy_y_right)
    split_info = calculate_entropy(total_zeros_and_ones_left, total_zeros_and_ones_right)

    return info_gain, split_info

In [314]:
def determine_candidate_splits(df):
    # candidate splits is a nested list containing [1, 0.04] i.e. 1 (for x1) and 0.04 for xj's value
    candidate_splits = []
    df_x1 = df.sort_values('x1')
    df_x1 = df_x1.reset_index(drop=True)
    df_x2 = df.sort_values('x2')
    df_x2 = df_x2.reset_index(drop=True)

    # total_number_of_zeros = df['y'].value_counts()[0]
    # total_number_of_ones = df['y'].value_counts()[1]
    total_number_of_zeros = np.sum(df['y'] == 0)
    total_number_of_ones = np.sum(df['y'] == 1)
    
    for i in range(1, len(df)):
        
        if df_x1['y'][i] != df_x1['y'][i - 1]:
            # y has changed
            number_of_zeros_left = np.sum(df_x1['y'].iloc[:i] == 0)
            number_of_ones_left = np.sum(df_x1['y'].iloc[:i] == 1)
            number_of_zeros_right = np.sum(df_x1['y'].iloc[i:] == 0)
            number_of_ones_right = np.sum(df_x1['y'].iloc[i:] == 1)
            # candidate_splits.append([1, df_x1['x1'][i], total_number_of_zeros, total_number_of_ones, number_of_zeros_left, number_of_ones_left, number_of_zeros_right, number_of_ones_right])
            info_gain, split_info = calculate_info_gain_and_split_info(total_number_of_zeros, total_number_of_ones, number_of_zeros_left, number_of_ones_left, number_of_zeros_right, number_of_ones_right)
            if split_info != 0.0:
                gain_ratio = info_gain / split_info
                candidate_splits.append([1, df_x1['x1'][i], info_gain, gain_ratio])
            
        if df_x2['y'][i] != df_x2['y'][i - 1]:
            # y has changed
            number_of_zeros_left = np.sum(df_x2['y'].iloc[:i] == 0)
            number_of_ones_left = np.sum(df_x2['y'].iloc[:i] == 1)
            number_of_zeros_right = np.sum(df_x2['y'].iloc[i:] == 0)
            number_of_ones_right = np.sum(df_x2['y'].iloc[i:] == 1)
            # candidate_splits.append([2, df_x2['x2'][i], total_number_of_zeros, total_number_of_ones, number_of_zeros_left, number_of_ones_left, number_of_zeros_right, number_of_ones_right])
            info_gain, split_info = calculate_info_gain_and_split_info(total_number_of_zeros, total_number_of_ones, number_of_zeros_left, number_of_ones_left, number_of_zeros_right, number_of_ones_right)
            if split_info != 0.0:
                gain_ratio = info_gain / split_info
                candidate_splits.append([2, df_x2['x2'][i], info_gain, gain_ratio])

    # We can have empty candidate splits if the df has 0 or 1 rows, or if all rows in df have same y,
    # or if none of the splits have non-0 split_info
    return candidate_splits

In [315]:
def stopping_criteria_satisfied(len_df, set_of_candidate_splits):
    # node is empty, or
    # all splits have zero gain ratio
    if len_df == 0:
        return True
        
    all_splits_have_zero_gain_ratio = True
    for candidate_split in set_of_candidate_splits:
        if len(candidate_split) >= 4 and candidate_split[3] != 0.0:
            all_splits_have_zero_gain_ratio = False
            break
    if all_splits_have_zero_gain_ratio:
        return True
    
    return False

In [316]:
class Node:
    # double check
    def __init__(self, feature_number, value, left_child, right_child):
        self.feature_number = feature_number
        self.value = value
        self.left_child = left_child
        self.right_child = right_child

In [317]:
def get_df_subsets(df, feature_number, feature_value):
    if feature_number == 1:
        df_sorted_on_x1 = df.sort_values('x1')
        df_sorted_on_x1 = df_sorted_on_x1.reset_index(drop=True)
        
        index_of_x1 = -1
        
        for i in range(0, len(df)):
            if df_sorted_on_x1['x1'][i] == feature_value:
                index_of_x1 = i
                break
            
        left_df_subset = df_sorted_on_x1.iloc[index_of_x1:].copy().reset_index(drop=True)
        right_df_subset = df_sorted_on_x1.iloc[:index_of_x1].copy().reset_index(drop=True)
        
        return left_df_subset, right_df_subset
    else:
        df_sorted_on_x2 = df.sort_values('x2')
        df_sorted_on_x2 = df_sorted_on_x2.reset_index(drop=True)
        
        index_of_x2 = -1

        for i in range(0, len(df)):
            if df_sorted_on_x2['x2'][i] == feature_value:
                index_of_x2 = i
                break

        left_df_subset = df_sorted_on_x2.iloc[index_of_x2:].copy().reset_index(drop=True)
        right_df_subset = df_sorted_on_x2.iloc[:index_of_x2].copy().reset_index(drop=True)
        
        return left_df_subset, right_df_subset

In [318]:
def determine_class_label(df):
    if len(df) == 0:
        return 1
    number_of_zeros = np.sum(df['y'] == 0)
    number_of_ones = np.sum(df['y'] == 1)
    if number_of_zeros > number_of_ones:
        return 0
    else:
        return 1

In [319]:
def make_subtree(df):
    
    candidate_splits = determine_candidate_splits(df)
    
    if stopping_criteria_satisfied(len(df), candidate_splits):
        
        # Make leaf node, and assign a class label to it
        class_label = determine_class_label(df)
        leaf_node = Node(None, class_label, None, None)
        
        return leaf_node
        
    else:
        
        # Make internal node, assign split condition and children
        feature_number, feature_value = find_best_split(candidate_splits)
        
        left_df_subset, right_df_subset = get_df_subsets(df, feature_number, feature_value)

        print("left_df_subset: ", left_df_subset)
        print("right_df_subset: ", right_df_subset)
        
        left_child = make_subtree(left_df_subset)
        right_child = make_subtree(right_df_subset)
        
        internal_node = Node(feature_number, feature_value, left_child, right_child)
        
        return internal_node

In [320]:
def print_preorder_traversal(node, height):
    if node:
        if node.feature_number != None:
            print("    " * height + "x" + str(node.feature_number) + " >= " + str(node.value))
        else:
            print("    " * height + "y = " + str(node.value))
        
        print_preorder_traversal(node.left_child, height + 1)
        print_preorder_traversal(node.right_child, height + 1)

In [321]:
df = pd.read_csv('test.txt', delim_whitespace=True, header = None)
df.columns = ['x1', 'x2', 'y']

df.reset_index()

root_node = make_subtree(df)

print_preorder_traversal(root_node, 0)

left_df_subset:       x1    x2  y
0  0.03  0.02  0
1  0.04  0.01  0
right_df_subset:       x1    x2  y
0  0.01  0.04  1
1  0.02  0.03  1
x1 >= 0.03
    y = 0
    y = 1
